In [57]:
import os
import re
from openai import OpenAI
import pandas as pd
from pandas import Timestamp, DataFrame
from datetime import timedelta

Save the OPENAI_KEY into ".env" file, and *** AND .env TO YOUR .gitignore FILE. ***

In [58]:
def convert_date(calendar_data):   
    # Convert 'start' and 'end' columns to datetime objects
    # calendar_data['start'] = pd.to_datetime(calendar_data['start'])
    # calendar_data['end']   = pd.to_datetime(calendar_data['end'])
    # Format 'start' and 'end' columns into the desired string format
    calendar_data['start'] = calendar_data['start'].dt.strftime('%Y-%m-%d %H:%M:%S')
    calendar_data['end']   = calendar_data['end'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    calendar_data['start'] = pd.to_datetime(calendar_data['start'])
    calendar_data['end']   = pd.to_datetime(calendar_data['end'])
    # calendar_data['attendees'] = [ str(item) for item in calendar_data['attendees'] ]
    # weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    # calendar_data['weekday'] =[ weekdays[pd.to_datetime(calendar_data["start"][i], format='%Y-%m-%d %H:%M:%S').weekday()] for i in range(len(calendar_data['start']))]
    
    return calendar_data

df_as_dicts = [
    {'ID': 'd119c9af-4d0c-4d08-a1b6-344d02557345', 'status': 'confirmed', 'summary': 'Planning meeting', 'start': Timestamp('2023-10-02 09:00:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-02 10:00:00-0700', tz='America/Los_Angeles'), 'duration': 3600.0, 'attendees': 'janesmith@example.com(needsAction)\njoshrock@example.com(needsAction)\nrichardbaker@example.com(needsAction)'},
    {'ID': 'a115b615-fd92-4397-865c-ddd367b8d3bd', 'status': 'confirmed', 'summary': 'Strategy review', 'start': Timestamp('2023-10-02 10:00:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-02 12:00:00-0700', tz='America/Los_Angeles'), 'duration': 7200.0, 'attendees': 'janesmith@example.com(needsAction)\nleonardhill@example.com(needsAction)\nrichardx@example.com(needsAction)\nrichardbaker@example.com(needsAction)'},
    {'ID': '4b3aae2a-193d-4fc4-957e-30fb429abe10', 'status': 'confirmed', 'summary': 'Lunch', 'start': Timestamp('2023-10-02 12:00:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-02 13:00:00-0700', tz='America/Los_Angeles'), 'duration': 3600.0, 'attendees': ''},
    {'ID': '79d54031-f8e9-4492-80bd-4fe34778e102', 'status': 'confirmed', 'summary': 'Manager 1 on 1', 'start': Timestamp('2023-10-02 13:30:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-02 14:00:00-0700', tz='America/Los_Angeles'), 'duration': 1800.0, 'attendees': 'richardx@example.com(needsAction)\nrichardbaker@example.com(needsAction)'},
    {'ID': '43872aa1-2904-4e6e-88a7-2d4a7ed62bb9', 'status': 'confirmed', 'summary': 'Interview with candidate: John', 'start': Timestamp('2023-10-02 14:00:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-02 15:00:00-0700', tz='America/Los_Angeles'), 'duration': 3600.0, 'attendees': 'john@externalcompany.com(needsAction)\nrichardbaker@example.com(needsAction)'},
    {'ID': '64de7fcc-448d-4445-8753-568eddfc47c1', 'status': 'confirmed', 'summary': 'Recruitment course', 'start': Timestamp('2023-10-02 16:00:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-02 17:30:00-0700', tz='America/Los_Angeles'), 'duration': 5400.0, 'attendees': 'hillary@recruitmentcourse.com(needsAction)\njanesmith@example.com(needsAction)\nrichardbaker@example.com(needsAction)\nterryterrence@example.com(needsAction)'},
    {'ID': 'b1a98166-7701-4f5f-82d0-ce9d4e3d1aa5', 'status': 'confirmed', 'summary': 'Interview with candidate: Sally', 'start': Timestamp('2023-10-03 09:30:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-03 10:00:00-0700', tz='America/Los_Angeles'), 'duration': 1800.0, 'attendees': 'sally@externalcompany.com(needsAction)\nrichardbaker@example.com(needsAction)'},
    {'ID': '5690101a-5a43-47ca-be3e-545f6ae13bc3', 'status': 'confirmed', 'summary': 'Interview with candidate: Brent', 'start': Timestamp('2023-10-03 10:00:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-03 10:30:00-0700', tz='America/Los_Angeles'), 'duration': 1800.0, 'attendees': 'brent@externalcompany.com(needsAction)\nrichardbaker@example.com(needsAction)'},
    {'ID': '833cc6ef-3ec8-4500-ba05-b7dc5a423f7a', 'status': 'confirmed', 'summary': 'Interview with candidate: Bob', 'start': Timestamp('2023-10-03 10:30:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-03 11:30:00-0700', tz='America/Los_Angeles'), 'duration': 3600.0, 'attendees': 'bob@externalcompany.com(needsAction)\nrichardbaker@example.com(needsAction)'},
    {'ID': '516ec6dc-15a4-452e-93d2-c8050bb5439e', 'status': 'confirmed', 'summary': 'Lunch and learn: ethical recruiting', 'start': Timestamp('2023-10-03 12:00:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-03 13:00:00-0700', tz='America/Los_Angeles'), 'duration': 3600.0, 'attendees': 'janesmith@example.com(needsAction)\nkensmith@example.com(needsAction)\nrichardbaker@example.com(needsAction)'},
    {'ID': '048df1ab-52f5-42d3-8c08-fd4665e83265', 'status': 'confirmed', 'summary': 'Interview with candidate: Alice', 'start': Timestamp('2023-10-03 13:00:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-03 13:30:00-0700', tz='America/Los_Angeles'), 'duration': 1800.0, 'attendees': 'alice@externalcompany.com(needsAction)\njanesmith@example.com(needsAction)\nrichardbaker@example.com(needsAction)'},
    {'ID': 'b9168d58-59a2-4fb1-b7d3-e364a3ab9dec', 'status': 'confirmed', 'summary': 'Interview with candidate: Jane', 'start': Timestamp('2023-10-03 13:30:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-03 14:00:00-0700', tz='America/Los_Angeles'), 'duration': 1800.0, 'attendees': 'jane@externalcompany.com(needsAction)\nrichardbaker@example.com(needsAction)'},
    {'ID': 'ffec807a-c51b-43bb-b9a8-f3bf31303dfb', 'status': 'confirmed', 'summary': 'Job fair: bob hill university', 'start': Timestamp('2023-10-03 14:30:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-03 16:00:00-0700', tz='America/Los_Angeles'), 'duration': 5400.0, 'attendees': ''},
    {'ID': 'e3e5aa3b-5646-41db-a9f0-6563fbd780ca', 'status': 'confirmed', 'summary': 'Recruitment analytics review', 'start': Timestamp('2023-10-03 16:30:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-03 17:00:00-0700', tz='America/Los_Angeles'), 'duration': 1800.0, 'attendees': 'janesmith@example.com(needsAction)\nrichardbaker@example.com(needsAction)'},
    {'ID': '2ee10fc7-88ef-4a3e-ad29-b0d4f52f355b', 'status': 'confirmed', 'summary': 'Online candidate search', 'start': Timestamp('2023-10-04 09:00:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-04 11:00:00-0700', tz='America/Los_Angeles'), 'duration': 7200.0, 'attendees': ''},
    {'ID': '5e5a1235-b1a4-449a-b2bf-a6604fa880e2', 'status': 'confirmed', 'summary': 'Interview with candidate: Stacy', 'start': Timestamp('2023-10-04 11:30:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-04 12:30:00-0700', tz='America/Los_Angeles'), 'duration': 3600.0, 'attendees': 'richardbaker@example.com(needsAction)\nstacy@externalcompany.com(needsAction)'},
    {'ID': '27215f76-ab8c-47c8-986f-fb92965e75c5', 'status': 'confirmed', 'summary': 'Lunch', 'start': Timestamp('2023-10-04 12:30:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-04 13:30:00-0700', tz='America/Los_Angeles'), 'duration': 3600.0, 'attendees': ''},
    {'ID': '39c217df-8138-4a9e-b219-f1af2574e738', 'status': 'confirmed', 'summary': 'Online candidate search', 'start': Timestamp('2023-10-04 14:00:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-04 16:00:00-0700', tz='America/Los_Angeles'), 'duration': 7200.0, 'attendees': ''},
    {'ID': 'bde5030b-698d-4378-abed-dd07a9621457', 'status': 'confirmed', 'summary': 'Job fair: alice smith university', 'start': Timestamp('2023-10-04 16:15:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-04 18:00:00-0700', tz='America/Los_Angeles'), 'duration': 6300.0, 'attendees': ''},
    {'ID': 'ee02de3c-502a-436f-9eda-7672570a80e3', 'status': 'confirmed', 'summary': 'Interview with candidate: Lenny', 'start': Timestamp('2023-10-05 09:30:00-0700', tz='America/Los_Angeles'), 'end': Timestamp('2023-10-05 10:00:00-0700', tz='America/Los_Angeles'), 'duration': 1800.0, 'attendees': 'richardbaker@example.com(needsAction)\nlenny@externalcompany.com(needsAction)'}
]
    
calendar_data = DataFrame(df_as_dicts)  
calendar_data = convert_date(calendar_data)
# calendar_data

For normal use GPT-4 test

In [150]:
PROMPT = """You are provided a calendar dataset. This calendar is a Pandas dataframe named filtered_calendar_data, columns = [ID, status, summary, start, end , duration, attendees].
    This DataFrame filtered_calendar_data is a filtered dataframe, which includes all of your meeting records you need to answer the question. For example, if the question is in which day I have most meetings this week, then the calendar_data only consists of this weeks meeting records.
    Provide a chain of thought for your reasoning, analyze the date requirement, filter dataframe, and then fill the python function <query_dataframe>. 
    First, based on the dataframe and question, breakdown the question into several steps; Second, implement those steps in Python using the datetime module and Pandas:

    The input dataframe filtered_calendar_data have following columns:
    - ID: meeting ID;
    - status: meeting status, including the following status: cancelled, confirmed, tentative, data type 'string'.
    - summary: meeting or event topic, data type 'string';
    - start: the start datetime of meeting, data type 'datetime64[ns]', and date format: '%Y-%m-%d %H:%M:%S'.;
    - end: the start datetime of meeting, data type 'datetime64[ns]', and date format: '%Y-%m-%d %H:%M:%S',;
    - duration: meeting duration, data type 'int64', and date format: second;
    - attendees: email address of people who attend the meeting, data type 'str', emails address are delimited by the line terminator.
    
    **instruction**
    - provided python packages:'pandas' and 'datetime';
    - the python function is <query_dataframe>;
    - the inpout and the Return value are all DataFrame;
    - the default parameter: assuming start of the day at "09:00:00" end of day at "18:00:00", the default event interval is 30 minutes; 
    - all the questions use same timezone, thus don't have to convert the timezone;
    - if the today's date is required, use the floowing python code:
    ```python
    # get today's date, if the question is related to datetime, use today_date, otherwise ignore this code
    today_date = pd.to_datetime('2023-10-02 08:25:30', format='%Y-%m-%d %H:%M:%S')
    ```
    _example_:
    **task**: Analyze my calendar and find out the same time slot every weekday to schedule regular meeting 
    **example**:
    ```python
    def query_dataframe(filtered_calendar_data):
        # Step 1: Filter out cancelled meetings
        active_meetings = filtered_calendar_data[filtered_calendar_data['status'] != 'cancelled']
        
        # Step 2: Extract day of the week
        active_meetings['day_of_week'] = active_meetings['start'].dt.dayofweek
        
        # Step 3: Creating 30-minute windows during business hours
        business_hours = pd.date_range(start="09:00:00", end="18:00:00", freq="30T")
        time_slots = [(start.time(), (start + pd.Timedelta(minutes=30)).time()) for start in business_hours]
        
        # Step 4: Filter for weekdays (Monday(0) to Friday(4))
        weekday_meetings = active_meetings[active_meetings['day_of_week'].isin(range(5))]
        
        # Step 5: Group by day of the week
        grouped_by_day = weekday_meetings.groupby("day_of_week")
        
        # Step 6: Analyzing consistency across weekdays
        consistent_slots = []
        for slot in time_slots:
            slot_coverage = 0
            for _, day_group in grouped_by_day:
                if any((day_group['start'].dt.time <= slot[0]) & (day_group['end'].dt.time >= slot[1])):
                    slot_coverage += 1
            
            if slot_coverage == 0:  # each weekday has at least one meeting covering the slot
                consistent_slots.append(slot)
        
        # Step 7: Results Generation
        if consistent_slots:
            return pd.DataFrame(consistent_slots, columns=["Start Time", "End Time"])
        else:
            return pd.DataFrame(columns=["Start Time", "End Time"])
    ```

    Here is the question: At what hour of the day I typically have the most meetings?
    """

In [151]:
MODEL = "gpt-4-turbo"
# MODEL = "gpt-3.5-turbo"

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content":PROMPT,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

To find the hour of the day during which you typically have the most meetings from the provided `filtered_calendar_data` DataFrame, you would generally follow these steps:

1. **Exclude Cancelled Meetings**: You do not want to include meetings that were cancelled in your analysis, since those meetings did not actually take place.

2. **Extract the Hour from Start Times**: You need to isolate the hour from the start times of all meetings. This will let you aggregate meetings by each hour of the day.

3. **Count Meetings per Hour**: Count the number of meetings that start within each distinct hour over the dataset. This step involves grouping the data by hour and counting instances.

4. **Identify the Hour with Maximum Frequency**: From the aggregation in the previous step, determine which hour has the highest count of meetings.

Implementing this in Python using Pandas might look like the following:

```python
import pandas as pd

def query_dataframe(filtered_calendar_data):
    # Step 

## Code test

In [128]:
def get_python_code(llm_reply_with_code):
    _python_code_re_pattern = "```python\n(.*?)```"
    llm_reply_without_code = re.sub(
                _python_code_re_pattern, "", llm_reply_with_code, flags=re.DOTALL
                )
    python_code_list = re.findall(_python_code_re_pattern, llm_reply_with_code, re.DOTALL)
    return python_code_list[0]

python_code_list = get_python_code(generated_text)
python_code_list

'import pandas as pd\n\ndef query_dataframe(filtered_calendar_data):\n    # Step 1: Filter out cancelled meetings\n    active_meetings = filtered_calendar_data[filtered_calendar_data[\'status\'] != \'cancelled\']\n    \n    # Step 2: Extract day of the week\n    active_meetings[\'day_of_week\'] = active_meetings[\'start\'].dt.dayofweek\n    \n    # Step 3: Generating 30 min time windows\n    business_hours = pd.date_range(start="09:00", end="18:00", freq="30T")\n    time_slots = [(start.time(), (start + pd.Timedelta(minutes=30)).time()) for start in business_hours]\n    \n    # Filtering relevant days\n    monday_tuesday_meetings = active_meetings[active_meetings[\'day_of_week\'].isin([0, 1])]\n    \n    # Step 4: Format meetings\' start times to nearest half hour\n    monday_tuesday_meetings[\'time_slot_start\'] = monday_tuesday_meetings[\'start\'].dt.floor(\'30T\').dt.time\n    \n    # Step 5: Analyze consistencies\n    consistent_slots = {}\n    for slot in time_slots:\n        temp

In [129]:
try:
    # define the generated <query_dataframe> function code 
    exec(python_code_list)
    answer = query_dataframe(calendar_data.copy())
except Exception as E:
    print(f"""Non-executable code: {E}""")
    pass

answer

<string>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Start Time,End Time
0,10:00:00,10:30:00
1,12:00:00,12:30:00
2,13:30:00,14:00:00


In [131]:
today_date = pd.to_datetime('2023-10-02 08:25:30', format='%Y-%m-%d %H:%M:%S')
today_date1 = pd.to_datetime('2023-10-02 09:25:30', format='%Y-%m-%d %H:%M:%S')

print(today_date<today_date1)

True
